In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/ジャンクション/顔画像をcos類似度で計算

/content/drive/My Drive/ジャンクション/顔画像をcos類似度で計算


In [0]:
ls

crop_face_images.ipynb   input_images/          vgg_face_weights.h5
cv_oneshot_images.ipynb  output_images/         WeightsOpenCV/
db_images/               predict_similar.ipynb  画像退避ファイル/


#openCVで画像内の顔検出後、ワンショット学習で人物特定を行う

In [0]:
import numpy as np
import cv2
import io,os

#入力画像ファイルディレクトリ
input_dir="./input_images"
#ディレクトリに格納されているファイル一覧取得
input_file_lists = os.listdir(input_dir)

#DB用の画像ファイルディレクトリ
db_dir="./db_images"
#ディレクトリに格納されているファイル一覧取得
db_file_lists = os.listdir(db_dir)

#加工後の画像ファイル出力先ディレクトリ
output_dir="./output_images"

#cv2重みファイル読み込み（githubからcloneでなくpipなどでインストールしたほうが確実）
cascade = cv2.CascadeClassifier("WeightsOpenCV/haarcascade_frontalface_alt2.xml")

#顔の傍に書き出す文字データ読み込み
font=cv2.FONT_HERSHEY_SIMPLEX
#顔が一つも検出されなかった画像の数カウンタ
n=0
#顔検出されなかった画像のリスト
n_files=[]

#顔認証（ワンショット学習）を行うモデル（VGG-Face）の読み込み

In [0]:
import pandas as pd
from collections import OrderedDict
from keras.models import Model,model_from_json,Sequential
from keras.layers import ZeroPadding2D,Conv2D,MaxPooling2D
from keras.layers.core import Activation,Dense,Flatten,Dropout
from keras.preprocessing.image import load_img, save_img, img_to_array, array_to_img
from keras.applications.vgg19 import preprocess_input

Using TensorFlow backend.


In [0]:
#モデル構造定義
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Conv2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [0]:
#重みファイル読み込み
model.load_weights("vgg_face_weights.h5")
#特徴抽出までの層を使う
vgg_face_descriptor=Model(inputs=model.layers[0].input,
                          outputs=model.layers[-2].output)

In [0]:
#DB画像を処理できるよう画像整形を行う関数
def preprocess_dbimage(image_path):
    #target_sizeにリサイズして読み込み
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [0]:
epsilon = 0.15 #cosine similarity（cos類似度の閾値、似ているほど0に近くなる形式）
#cos類似度を求める関数（verifyFace()内で呼ばれる）
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [0]:
#ワンショット学習で人物特定する関数
def verifyFace(crop_img, db_img_path):
  crop_img_representation = vgg_face_descriptor.predict(crop_img)[0,:]
  db_img_representation = vgg_face_descriptor.predict(preprocess_dbimage(db_img_path))[0,:]
 
  cosine_similarity = findCosineDistance(crop_img_representation, db_img_representation)
  return  cosine_similarity


#画像解析
googlecoraboratoryGPU環境・・・平均1枚につき2秒くらい、検出される顔が少ないほど早い<br>
googlecoraboratoryCPU環境・・・1枚につき最速10秒、遅いと20秒くらい処理に時間かかる。
<br>
人物特定して配列に結果を格納する時間と、<br>
unkwonによるモザイク処理、どちらに時間かかるは未検証<br>
<br>
（モザイク処理入れるとGPUで1秒くらい遅くなった）

In [0]:
for input_file_name in input_file_lists:
    root, ext = os.path.splitext(input_file_name)
    if ext == u'.png' or u'.jpeg' or u'.jpg':
      abs_name = input_dir  + '/'+input_file_name
      #画像ファイルパスから画像の読み込み
      img = cv2.imread(abs_name,cv2.IMREAD_COLOR)
      #画像のグレースケール化
      #img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      #検出された顔らの情報がfaceに入る
      face=cascade.detectMultiScale(img,scaleFactor=1.1, minNeighbors=3)
      color = (255, 255, 255) #検出範囲を囲う線の色を指定
      print("検出対象ファイル："+input_file_name)
      print("検出された顔の数："+str(len(face)))
      #検出された顔カウンタ
      i=1
      #検出範囲の枠を書き込んだ画像の保存先パス
      output_image_path =output_dir +'/'+root+'_output' + ext
      if len(face) > 0:  
        for rect in face:
          x=rect[0]
          y=rect[1]
          width=rect[2]
          height=rect[3]
          #dst_imgが検出された顔領域
          dst_img=img[y:y+height,x:x+width]
          
          #検出された顔画像の前処理
          dst=cv2.resize(dst_img,(224,224))
          dst = img_to_array(dst)
          dst = np.expand_dims(dst, axis=0)
          dst = preprocess_input(dst)
          
          #検出された顔毎の人物特定結果配列（名前とスコアが、keyとvalueの関係）
          result_score = OrderedDict()
          #dbにある画像全てと比較
          for db_file_name in db_file_lists:
            #画像の類似度判定
            score=verifyFace(dst,db_dir+'/'+db_file_name)
            #閾値によるチェック
            if score < epsilon:
              result_score[db_file_name.split('.')[0]]=score
          
          #類似顔が1つもないとき
          if len(result_score)==0:
        
            #モザイク処理
            moz_face = dst_img.shape[:2][::-1]
            #10分の1にする
            #dst_img = cv2.resize(dst_img,(int(moz_face[0]/10),int(moz_face[0]/10)))
            dst_img = cv2.resize(dst_img,(moz_face[0]//10,moz_face[0]//10))
            #画像を元のサイズに拡大
            dst_img = cv2.resize(dst_img,moz_face,interpolation = cv2.INTER_NEAREST)
            #モザイク処理した部分を重ねる
            img[y:y+height,x:x+width] = dst_img
            
            
            #元画像に解析結果出力
            cv2.putText(img,'unkown',tuple(rect[0:2]), font, 1,(0,0,0),2,cv2.LINE_AA)
          else:
            #類似度順（cosine_similarityの昇順）に並び替え
            result_score = OrderedDict(sorted(result_score.items(), key=lambda x: x[1], reverse=False))
            print(result_score)
            '''
            #特定人物用モザイク処理
            if str(next(iter(result_score)))=="人物名指定":
              moz_face = dst_img.shape[:2][::-1]
              #10分の1にする
              #dst_img = cv2.resize(dst_img,(int(moz_face[0]/10),int(moz_face[0]/10)))
              dst_img = cv2.resize(dst_img,(moz_face[0]//10,moz_face[0]//10))
              #画像を元のサイズに拡大
              dst_img = cv2.resize(dst_img,moz_face,interpolation = cv2.INTER_NEAREST)
              #モザイク処理した部分を重ねる
              img[y:y+height,x:x+width] = dst_img
            '''
            #元画像に解析結果出力
            cv2.putText(img,str(next(iter(result_score)))+':'+str(next(iter(result_score.values())))[0:5],tuple(rect[0:2]), font, 0.5,(0,255,0),2,cv2.LINE_AA)
           
          #元画像に検出範囲の枠を書き込む
          cv2.rectangle(img,tuple(rect[0:2]),tuple(rect[0:2]+rect[2:4]),color,thickness=2)
          del result_score
          
        cv2.imwrite(output_image_path,img)
      else:
        print("no face")
        n+=1
        n_files.append(input_file_name)
        cv2.imwrite(output_image_path,img)
               

検出対象ファイル：keyaki2.jpg
検出された顔の数：8
OrderedDict([('nagahama', 0.1397119164466858)])
検出対象ファイル：shuugou1.jpg
検出された顔の数：3
検出対象ファイル：keyaki1.jpg
検出された顔の数：4
OrderedDict([('nagahama', 0.12542986869812012)])
検出対象ファイル：keyaki3.jpg
検出された顔の数：6
OrderedDict([('nagahama', 0.14215201139450073)])
OrderedDict([('hirate', 0.03329622745513916)])
